In [37]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import pandas as pd
import numpy as np 
from sklearn.preprocessing import StandardScaler
import tensorflow_probability as tfp
from causallearn.utils.KCI.KCI import KCI_CInd
from causallearn.utils.cit import kci
import waterbirds.data_builder as db
import functools
from PyRKHSstats.kcit import perform_kcit, ImplementedKCITSchemes
from scipy.spatial.distance import pdist
from PyRKHSstats.kernel_wrapper import KernelWrapper
from sklearn.gaussian_process.kernels import RBF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# for rs in [i for i in range(10)]:
    
#     # ---- import data
#     experiment_directory = (f'/data/ddmg/scate/multiple_shortcut/waterbirds/experiment_data/'
#         f'rs{rs}')
#     tr_data = pd.read_csv(f'{experiment_directory}/train.txt')
    
#     tr_data = tr_data['0'].str.split(",", expand=True)
#     D = tr_data.shape[1]-4
#     tr_data.columns = ['bird_img', 'bird_seg', 'back_img', 'noise_img'] + \
#         [f'y{i}' for i in range(D)]
#     tr_data.drop(['bird_img', 'bird_seg', 'back_img', 'noise_img'], axis=1, inplace=True)
#     for i in range(D):
#         tr_data[f'y{i}'] = tr_data[f'y{i}'].astype(np.float32)
        
#     D = 13
#     ys = StandardScaler().fit_transform(tr_data.y0.values.reshape(-1, 1))
#     hsic_vals = []
#     for i in range(1, D):
#         vs = StandardScaler().fit_transform(tr_data[[f'y{j}' for j in range(1, D) if j !=i ]])
#         sigma = np.float(1 / np.sqrt(1/(D-2.0)))
#         hsic_vals.append(hsic(vs, ys, sigma=sigma, sample_weights=None)[0].numpy())
#     hsic_vals = sorted(hsic_vals)
#     pct_change = 100* (hsic_vals[-1] - hsic_vals[-2])/hsic_vals[-2]
#     print(f'RS {rs}, % change {pct_change}')
    

In [60]:
# --- just the outcome
for rs in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
    # ---- import data
    experiment_directory = (f'/data/ddmg/scate/multiple_shortcut/waterbirds/experiment_data/'
        f'rs{rs}')
    tr_data = pd.read_csv(f'{experiment_directory}/train.txt')
    tr_data = tr_data['0'].str.split(",", expand=True)
    D = tr_data.shape[1]-4
    tr_data.columns = ['bird_img', 'bird_seg', 'back_img', 'noise_img'] + \
        [f'y{i}' for i in range(D)]
    tr_data.drop(['bird_img', 'bird_seg', 'back_img', 'noise_img'], axis=1, inplace=True)
    for i in range(D):
        tr_data[f'y{i}'] = tr_data[f'y{i}'].astype(np.float32)
    
    
    D = 13
    p_values = []
    for i in range(1, D):
        
        kci_obj = KCI_CInd(kernelX='Gaussian', 
                   kernelY='Gaussian', 
                   kernelZ='Gaussian',
                   est_width='median', 
                   epsilon=1e-3)
    
        p, _ = kci_obj.compute_pvalue(data_x=tr_data.values[:, 0][:, np.newaxis],
                                  data_y=tr_data.values[:, i][:, np.newaxis], 
                                  data_z = tr_data.values[:,[j for j in range(D) if ((j!=0) & (j!=i))]]
            )

        p_values.append(p)
        print(f'P {i}, {np.round(p_values[-1], 3)}')
    

P 1, 0.0
P 2, 0.0
P 3, 0.006
P 4, 0.19
P 5, 0.02
P 6, 0.0
P 7, 0.655
P 8, 0.781
P 9, 0.049
P 10, 0.001
P 11, 0.093
P 12, 0.054
P 1, 0.0
P 2, 0.0
P 3, 0.003
P 4, 0.02
P 5, 0.08
P 6, 0.015
P 7, 0.013
P 8, 0.047
P 9, 0.327
P 10, 0.284
P 11, 0.537
P 12, 0.469
P 1, 0.0
P 2, 0.0
P 3, 0.281
P 4, 0.363
P 5, 0.285
P 6, 0.001
P 7, 0.33
P 8, 0.893
P 9, 0.475
P 10, 0.06
P 11, 0.06
P 12, 0.007
P 1, 0.0
P 2, 0.0
P 3, 0.006
P 4, 0.032
P 5, 0.016
P 6, 0.553
P 7, 0.007
P 8, 0.313
P 9, 0.043
P 10, 0.117
P 11, 0.21
P 12, 0.057
P 1, 0.0
P 2, 0.0
P 3, 0.412
P 4, 0.517
P 5, 0.116
P 6, 0.022
P 7, 0.585
P 8, 0.304
P 9, 0.668
P 10, 0.111
P 11, 0.009
P 12, 0.155
P 1, 0.0
P 2, 0.0
P 3, 0.208
P 4, 0.085
P 5, 0.062
P 6, 0.001
P 7, 0.001
P 8, 0.429
P 9, 0.005
P 10, 0.008
P 11, 0.306
P 12, 0.371
P 1, 0.0
P 2, 0.0
P 3, 0.27
P 4, 0.618
P 5, 0.16
P 6, 0.3
P 7, 0.685
P 8, 0.05
P 9, 0.057
P 10, 0.003
P 11, 0.31
P 12, 0.007
P 1, 0.0
P 2, 0.0
P 3, 0.477
P 4, 0.141
P 5, 0.004
P 6, 0.08
P 7, 0.163
P 8, 0.824
P 9, 0.067
P 10,

In [4]:


def calc_kci(data, x_cols, y_cols, z_cols, 
             test_level = 0.001, epsilon=1e-3):
    data_x = data[x_cols].values
    data_y = data[y_cols].values
    data_z = data[z_cols].values

    data_x = StandardScaler().fit_transform(data_x)
    data_y = StandardScaler().fit_transform(data_y)
    data_z = StandardScaler().fit_transform(data_z)
    
    # Kernels to use
    length_scale_kx = np.median(np.abs(pdist(data_x)))
    if length_scale_kx < 1e-3:
        length_scale_kx = 0.01
    kernel_kx = KernelWrapper(RBF(length_scale=length_scale_kx))
    length_scale_ky = np.median(np.abs(pdist(data_y)))
    if length_scale_ky < 1e-3:
        length_scale_ky = 0.01
        
    kernel_ky = KernelWrapper(RBF(length_scale=length_scale_ky))
    length_scale_kz = np.median(np.abs(pdist(data_z)))
    if length_scale_kz < 1e-3:
        length_scale_kz = 0.01
        
    kernel_kz = KernelWrapper(RBF(length_scale=length_scale_kz))

    res = perform_kcit(data_x, data_y, data_z, kernel_kx, kernel_ky, kernel_kz,
                 epsilon=epsilon,
                       test_level=test_level, scheme=ImplementedKCITSchemes['MONTECARLO']
            )
    print(res)
    return res['Reject H0 (H0 : X _||_ Y | Z)']


In [40]:
# best case scenario

for rs in [0]:
    # ---- import data
    experiment_directory = (f'/data/ddmg/scate/multiple_shortcut/chexpert/experiment_data/'
        f'rs{rs}')
    tr_data = pd.read_csv(f'{experiment_directory}/dag1_train.txt')
    tr_data = tr_data['0'].str.split(",", expand=True)
#     D = tr_data.shape[1]-4
#     tr_data.columns = ['bird_img', 'bird_seg', 'back_img', 'noise_img'] + \
#         [f'y{i}' for i in range(D)]
#     tr_data.drop(['bird_img', 'bird_seg', 'back_img', 'noise_img'], axis=1, inplace=True)
    D = tr_data.shape[1]-2
    tr_data.columns = ['img_filename', 'noise_filename'] + \
        [f'y{i}' for i in range(D)]
    tr_data.drop(['img_filename', 'noise_filename'], axis=1, inplace=True)

    
    for i in range(D):
        tr_data[f'y{i}'] = tr_data[f'y{i}'].astype(np.float32)
    
    preds = tr_data[[f'y{i}' for i in range(1, D)]]
    preds['y1'] = preds['y1'] * np.random.uniform(0.8, 0.9, preds.shape[0]) + \
        (1.0 - preds['y1']) * np.random.uniform(0.1, 0.3, preds.shape[0])
    
    preds['y2'] = preds['y2'] * np.random.uniform(0.8, 0.9, preds.shape[0]) + \
    (1.0 - preds['y2']) * np.random.uniform(0.1, 0.3, preds.shape[0])
    
    preds[[f'y{i}' for i in range(3, D)]] = preds[
        [f'y{i}' for i in range(3, D)]] * np.random.uniform(0.4, 0.6, 
        preds[[f'y{i}' for i in range(3, D)]].shape) + \
        (1- preds[[f'y{i}' for i in range(3, D)]]) * np.random.uniform(0.3, 0.6,                                           
        preds[[f'y{i}' for i in range(3, D)]].shape)
    
    preds.columns = [f'p{i}' for i in range(1, D)]
    tr_data = pd.concat([tr_data, preds], axis=1)
    pred_idx = [tr_data.columns.tolist().index(col) for col in tr_data if col.startswith('p')]
    var_idx = [i for i in range(1, D)]
    D = 13
    p_values = []
    
    tr_data = tr_data.sample(frac=0.5)


for i in range(1, D):
    p = calc_kci(tr_data, [f'p{i}' for i in range(1, D)],
                 [f'y{i}'], [f'y{j}' for j in range(D) if (j!=i)], 
                epsilon = 1e-3, test_level=0.05)
    p_values.append(p)
    print(f'col {i}, {p_values}')


KeyboardInterrupt: 

In [51]:
def get_train_data(random_seed, base_dir, pixel=128, weighted=False, batch_size=100):
    """Function to get the data."""
    experiment_directory = (
        f"{base_dir}/experiment_data/rs{random_seed}")
    train_data = pd.read_csv(
        f'{experiment_directory}/train.txt'
        ).values.tolist()

    train_data = [
        tuple(train_data[i][0].split(',')) for i in range(len(train_data))
    ]
        
    
    
    map_to_image_label_given_pixel = functools.partial(db.map_to_image_label,
            pixel=pixel, weighted=weighted)

    train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
    train_dataset = train_dataset.map(map_to_image_label_given_pixel, num_parallel_calls=1)
    train_dataset = train_dataset.batch(batch_size,
        drop_remainder=False).repeat(1)
    return train_dataset

In [61]:

rs = 3
base_dir = '/data/ddmg/scate/multiple_shortcut/waterbirds'
batch_size = 1000 
pixel = 128
train_dataset = get_train_data(
    random_seed=rs,
    base_dir=base_dir, 
    batch_size = batch_size, 
    pixel = pixel
)

tr_data = []
for batch_id, examples in enumerate(train_dataset):
    print(batch_id)
    x, labels_weights = examples
    x = x.numpy()
    x = x.reshape(x.shape[0], pixel*pixel*3)
    pix_cols = [f'pix{i}' for i in range(x.shape[1])]
    labels = labels_weights['labels'].numpy()
    label_cols = [f'y{i}' for i in range(labels.shape[1])]
    x = pd.DataFrame(np.hstack([labels, x]), 
                    columns = label_cols + pix_cols)
    tr_data.append(x)

tr_data = pd.concat(tr_data)





0
1


In [62]:
# tr_data_orig = tr_data.copy()
# tr_data = tr_data.sample(frac=0.5)

In [64]:
D = 13
p_values = []
pix_idx = list(set(range(tr_data.shape[1])) - set(range(13)))
for i in range(1, D):
#     p = calc_kci(tr_data, [col for col in tr_data if col.startswith("pix")],
#                  [f'y{i}'], [f'y{j}' for j in range(D) if (j!=i)], 
#                 epsilon = 1e-3, test_level = 0.05)
#     p_values.append(p)
    kci_obj = KCI_CInd(kernelX='Gaussian', 
                       kernelY='Gaussian', 
                       kernelZ='Gaussian',
                       est_width='median', 
                       epsilon=1e-3)
    
    p, _ = kci_obj.compute_pvalue(data_x=tr_data.values[:, pix_idx],
                              data_y=tr_data.values[:, i][:, np.newaxis], 
                              data_z = tr_data.values[:,[j for j in range(D) if (j!=i)]]
        )
    p_values.append(p)
    
    print(f'col {i}, {p_values}')

col 1, [0.0]
col 2, [0.0, 0.044015169908673046]
col 3, [0.0, 0.044015169908673046, 0.4038164557875653]
col 4, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556]
col 5, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718]
col 6, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718, 0.4282745657571312]
col 7, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718, 0.4282745657571312, 0.40032589729544954]
col 8, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718, 0.4282745657571312, 0.40032589729544954, 0.6016109101492958]
col 9, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718, 0.4282745657571312, 0.40032589729544954, 0.6016109101492958, 0.3188263963897868]
col 10, [0.0, 0.044015169908673046, 0.4038164557875653, 0.3923169993215556, 0.2996813627991718, 0.4282745657571312, 0.40032589729544954, 0.60161

In [36]:
tr_data.shape

(1119, 49165)

In [35]:
for i in range(1, D):
    if i > 1:
        break
    p = kci(data=tr_data.values, X=pix_idx, Y=[i], 
    condition_set = [j for j in range(D) if (j!=i)], 
    kernelX='Gaussian', 
    kernelY='Gaussian',
    kernelZ='Gaussian',
    est_width='empirical')
    p_values.append(p)
    print(f'Excluded variable {i}, {p_values[-1]}')
    

X
[0.39970198 0.39970198 0.39970198 0.3770019  0.3770019  0.3770019
 0.61781335 0.61781335 0.61781335 0.5814944  0.5814944  0.5814944 ]
Y
[0.53]
condition
[0.2625 0.445  0.0075 0.02   0.0125 0.02   0.0125 0.015  0.005  0.0075
 0.0075 0.01  ]
Excluded variable 1, 2.2548467447536247e-05


In [36]:
p = kci(data=tr_data.values, X=pix_idx, Y=[1, 2], 
condition_set = [j for j in range(D) if ((j!=1) & (j!=2)) ], 
kernelX='Gaussian', 
kernelY='Gaussian',
kernelZ='Gaussian',
est_width='empirical')
p_values.append(p)
print(f'Excluded variable {i}, {p_values[-1]}')

X
[0.39970198 0.39970198 0.39970198 0.3770019  0.3770019  0.3770019
 0.61781335 0.61781335 0.61781335 0.5814944  0.5814944  0.5814944 ]
Y
[0.53  0.445]
condition
[0.2625 0.0075 0.02   0.0125 0.02   0.0125 0.015  0.005  0.0075 0.0075
 0.01  ]
Excluded variable 2, 0.00011928655798865151


In [10]:
tr_data.mean(axis=0)

y0       0.255250
y1       0.553250
y2       0.463000
y3       0.007750
y4       0.020750
y5       0.016250
y6       0.009000
y7       0.013500
y8       0.010750
y9       0.005750
y10      0.008500
y11      0.010000
y12      0.005750
pix0     0.408232
pix1     0.408232
pix2     0.408232
pix3     0.376849
pix4     0.376849
pix5     0.376849
pix6     0.612057
pix7     0.612057
pix8     0.612057
pix9     0.581886
pix10    0.581886
pix11    0.581886
dtype: float32

In [17]:
tr_data['pix1']  = np.random.choice(tr_data['pix1'], size = tr_data.shape[0], replace=False)

In [23]:
tr_data[
    [col for col in tr_data.columns.tolist() if col.startswith('pix')]] = np.random.uniform(
0.0, 1.0, size = tr_data[
    [col for col in tr_data.columns.tolist() if col.startswith('pix')]].shape)

In [24]:
for i in range(1, D):
    if i > 4:
        break
    p = kci(data=tr_data.values, X=[13], Y=[i], 
    condition_set = None,  
    kernelX='Gaussian', 
    kernelY='Gaussian',
    kernelZ='Gaussian',
    est_width='empirical')
    p_values.append(p)
    print(f'Excluded variable {i}, {p_values[-1]}')

Excluded variable 1, 0.7818504230861517
Excluded variable 2, 0.18980220115769386
Excluded variable 3, 0.6718491286387864
Excluded variable 4, 0.664530478589391


In [11]:
tr_data.to_csv("/data/ddmg/scate/scratch/temp.csv")

In [22]:
from PyRKHSstats.kcit import perform_kcit, ImplementedKCITSchemes
from scipy.spatial.distance import pdist
from PyRKHSstats.kernel_wrapper import KernelWrapper
from sklearn.gaussian_process.kernels import RBF

<ImplementedKCITSchemes.GAMMA: 'Gamma Approximation'>

In [44]:
data_x = tr_data[[col for col in tr_data.columns if col.startswith("pix")]].values
data_x = StandardScaler().fit_transform(data_x)

data_y = tr_data[['y5']].values
data_y = StandardScaler().fit_transform(data_y)

data_z = tr_data[[col for col in tr_data.columns if col.startswith("y")]]
data_z.drop("y5", inplace =True, axis=1)
data_z = data_z.values 
data_z = StandardScaler().fit_transform(data_z)


# Kernels to use
length_scale_kx = np.median(np.abs(pdist(data_x)))
kernel_kx = KernelWrapper(RBF(length_scale=length_scale_kx))
length_scale_ky = np.median(np.abs(pdist(data_y)))
kernel_ky = KernelWrapper(RBF(length_scale=length_scale_ky))
length_scale_kz = np.median(np.abs(pdist(data_z)))
kernel_kz = KernelWrapper(RBF(length_scale=length_scale_kz))

    

In [66]:
perform_kcit(data_x, data_x, data_z, kernel_kx, kernel_kx, kernel_kz,
                 epsilon=1.0, test_level=0.001, scheme=ImplementedKCITSchemes['MONTECARLO']
            )

{'Reject H0 (H0 : X _||_ Y | Z)': False,
 'TCI': 0.2876415752768831,
 'Rejection threshold': 1.0009759570211545,
 'Samples from simulated null': array([[0.27053807],
        [0.08694789],
        [0.20141119],
        ...,
        [0.30113187],
        [0.10382512],
        [0.17244558]])}

In [55]:
tr_data['pix1'][(tr)]

0      0.509804
1      0.640196
2      0.141176
3      0.055392
4      0.125490
         ...   
995    0.195098
996    0.530392
997    0.484314
998    0.451225
999    0.521569
Name: pix1, Length: 4000, dtype: float32

In [28]:
tr_data.values[:, 13:]

array([[0.50980395, 0.50980395, 0.50980395, ..., 0.78333336, 0.78333336,
        0.78333336],
       [0.6401961 , 0.6401961 , 0.6401961 , ..., 0.03431373, 0.03431373,
        0.03431373],
       [0.14117648, 0.14117648, 0.14117648, ..., 0.73137254, 0.73137254,
        0.73137254],
       ...,
       [0.40098038, 0.40098038, 0.40098038, ..., 0.845098  , 0.845098  ,
        0.845098  ],
       [0.64117646, 0.64117646, 0.64117646, ..., 0.6431373 , 0.6431373 ,
        0.6431373 ],
       [0.5627451 , 0.5627451 , 0.5627451 , ..., 0.27058825, 0.27058825,
        0.27058825]], dtype=float32)

In [27]:
tr_data

,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,pix2,pix3,pix4,pix5,pix6,pix7,pix8,pix9,pix10,pix11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.509804,0.050000,0.050000,0.050000,0.757843,0.757843,0.757843,0.783333,0.783333,0.783333
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.640196,0.504902,0.504902,0.504902,0.260784,0.260784,0.260784,0.034314,0.034314,0.034314
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.141176,0.196078,0.196078,0.196078,0.509804,0.509804,0.509804,0.731373,0.731373,0.731373
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.055392,0.171569,0.171569,0.171569,0.825490,0.825490,0.825490,0.883578,0.883578,0.883578
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.125490,0.125490,0.125490,0.125490,0.737255,0.737255,0.737255,0.376471,0.376471,0.376471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.411765,0.156863,0.156863,0.156863,0.800000,0.800000,0.800000,0.847059,0.847059,0.847059
547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.531373,0.566667,0.566667,0.566667,0.807843,0.807843,0.807843,0.785294,0.785294,0.785294
548,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.400980,0.200000,0.200000,0.200000,0.851961,0.851961,0.851961,0.845098,0.845098,0.845098
549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.641176,0.237255,0.237255,0.237255,0.811765,0.811765,0.811765,0.643137,0.643137,0.643137


In [8]:
tr_data_or = tr_data.copy()

In [10]:
tr_data = tr_data_or.sample(frac=0.1)

In [13]:
for i in range(1, D):
    p = kci(data=tr_data.values, X=pix_idx, Y=[i], 
    condition_set = [j for j in range(D) if (j!=i)], 
    kernelX='Gaussian', 
    kernelY='Gaussian',
    kernelZ='Gaussian',
    est_width='empirical')
    p_values.append(p)
    print(f'Excluded variable {i}, {p_values[-1]}')

Excluded variable 1, 0.0
Excluded variable 2, 0.0
Excluded variable 3, 0.4622323343488939


KeyboardInterrupt: 

In [7]:
data_x = np.random.normal(0, 1, (10, 1))
data_y = np.random.normal(0, 1, (10, 1))
data_z = np.random.normal(0, 1, (10, 1))


# Kernels to use
length_scale_kx = np.median(np.abs(pdist(data_x)))
kernel_kx = KernelWrapper(RBF(length_scale=length_scale_kx))
length_scale_ky = np.median(np.abs(pdist(data_y)))
kernel_ky = KernelWrapper(RBF(length_scale=length_scale_ky))
length_scale_kz = np.median(np.abs(pdist(data_z)))
kernel_kz = KernelWrapper(RBF(length_scale=length_scale_kz))


perform_kcit(data_x, data_x, data_z, kernel_kx, kernel_kx, kernel_kz,
                 epsilon=1.0, test_level=0.001, scheme=ImplementedKCITSchemes['GAMMA']
            )

{'Reject H0 (H0 : X _||_ Y | Z)': False,
 'TCI': 0.4269226783830667,
 'Rejection threshold': 1.1793853331569217,
 'Gamma distribution': <scipy.stats._distn_infrastructure.rv_frozen at 0x7f30ddf97c10>}